# LastFM API Data Retrieval

> Build the LastFmAPI class. 

Use the class to pull recent tracks from Last FM, and include only those tracks above a certain playcount threshold.

In [ ]:
#| default_exp retrieve_last_fm_data

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import pandas as pd
import requests
import boto3
import json

In [ ]:
#| export
class LastFmAPI:
    """
    Provides an interface to the Last.fm API for retrieving top tracks for a user.

    Parameters
    ----------
    region_name : str
        The name of the AWS region where the secrets manager is located.
    """
    required_env_keys = ['spot_clientID', 'spot_clientSECRET', 'spot_ACC', 'spot_REF']

    def __init__(self, region_name):
        """
        Initializes a new instance of the LastFmAPI class.

        Parameters
        ----------
        region_name : str
            The name of the AWS region where the secrets manager is located.
        """
        self.region_name = region_name
    
    def get_secret(self, secret_name):
        """
        Retrieves the specified secret from AWS Secrets Manager and sets the corresponding environment variables.

        Parameters
        ----------
        secret_name : str
            The name of the secret to retrieve.
        """
        if all([k in os.environ.keys() for k in self.required_env_keys]):
            print('All required environment variables are set')
        else:
            session = boto3.session.Session()
            client = session.client(
                service_name='secretsmanager',
                region_name=self.region_name
            )
            secret_value = client.get_secret_value(
                SecretId=secret_name
            )
            secret_dict = json.loads(secret_value['SecretString'])
            for k, v in secret_dict.items():
                os.environ[k] = v

    def get_top_tracks(self, period='1month', playcount_cutoff=1):
        """
        Retrieves the top tracks for the user.

        Parameters
        ----------
        period : str, optional
            The time period for which to retrieve the top tracks. Default is '1month'.
        playcount_cutoff : int, optional
            The minimum number of plays required for a track to be included in the results. Default is 1.

        Returns
        -------
        pandas.DataFrame
            A DataFrame containing the top tracks for the user. The DataFrame has columns for the track name, artist name, and play count.
        """
        headers = {
        'user-agent': os.environ.get('last_userAGENT')
        }

        payload = {'method': 'user.gettoptracks', 
                    'period': period,
                    'user': os.environ.get('last_username'),
                    'api_key': os.environ.get('last_apiKEY'),
                    'format': 'json'
                    }


        r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)

        # Apply formatting
        tracks = r.json()['toptracks']['track']
        tracks = pd.DataFrame(tracks)
        tracks = tracks[['name', 'artist', 'playcount']]
        tracks['artist'] = tracks['artist'].apply(lambda x: x['name'])
        tracks['playcount'] = tracks['playcount'].astype(int)

        # Filter out tracks with low playcount
        tracks = tracks.sort_values('playcount', ascending=False)
        tracks = tracks[tracks['playcount'] >= playcount_cutoff]

        return tracks


In [ ]:
show_doc(LastFmAPI.get_secret)

In [ ]:
show_doc(LastFmAPI.get_top_tracks)

In [ ]:
#| export
if __name__ == '__main__':
    last = LastFmAPI('us-east-1')
    last.get_secret('last_keys')
    tracks = last.get_top_tracks()
    tracks.to_csv('s3://spotify-net/last_fm_recent_tracks.csv')
    print('Retrieved')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()